# A little bit of analysis goes a long way

As great as looking at raw data is, it is not our ultimate goal.  Reminding our selves, why we are (hypothetically) doing this: we want to determine how *control* affects the physical properties of a cantilever.  We know that we can model this system as a damped harmonic oscillator so the displacement away from equilibrium as a function of time is given by:

$$D(t) = A e^{-\zeta\omega_0t} \sin\left(\sqrt{1 - \zeta^2}\omega_0t + \varphi\right)$$

where $A$ and $\varphi$ are set by the initial conditions, $\zeta$ is the damping factor and $\omega_0$ is the natural frequency of the oscillator.

## (non-linear) fitting

In [ ]:
# not sure how else to get the helpers on the path!
import sys
sys.path.append('../scripts')

In [ ]:
from data_gen import get_data, fit, Params
d = get_data(25)
m = d[6]

In order to extract the fit parameters from our (noisy) data we will use the [`scipy.optimize.curve_fit`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html).  In the interest of time, we have implemented this in the helper module and will import it.

In the same spirit as our `plot_one` method, the `fit` function wraps the underlying `curve_fit` function and provides 2 improvements for our purposes:

1. Specializes from general non-linear fitting to "damped harmonic fitting" by baking in the functional form we know we want to fit
2. Enriches the returned parameters from a numpy array to a `namedtuple` with nice reprs and a `sample` method.

### TODO

Detail design review of `fit` and `Params`

In [ ]:
fit(m)

## Add fit to plot

In [ ]:
# interactive figures, requires ipypml!
%matplotlib widget
#%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import xarray as xa

In [ ]:
def plot_one(
    ax: "Axes",
    experiment: "OneExperiment",
    fit: Params,
    offset: float = 0,
    *,
    line_style: "Dict[str, Any]" = None,
) -> "Dict[str, Artist]":
    """Given a curve plot it (with an offset) and format a label for a legend.

    Parameters
    ----------
    ax : mpl.Axes
        The axes to add the plot to

    experiment : OneExperiment
        An xarray DataArray with a vector 'time' and scalar 'control' coordinates.
        
    fit : Params
        The fit Parameters

    offset : float, optional
        A vertical offset to apply to before plotting

    line_style : dict, optional
        Any any keywords that can be passed to `matplotlib.axes.Axes.plot`

    Returns
    -------
    curve : Line2D
        The Line2D object for the curve
    """
    # if the user does not give us input, 
    line_style = line_style if line_style is not None else {}
    # if the user passes label in the line_style dict, let it win!
    line_style.setdefault("label", f"control = {float(experiment.control):.1f}")
    # do the plot!
    t = experiment.time
    (ln,) = ax.plot(t, experiment + offset, **line_style)
    (fit_ln,) = ax.plot(t, fit.sample(t) + offset, color="k")
    ann = ax.annotate(
        f"$\\zeta={fit.zeta:.2g}$, $\\omega_0={fit.omega:.2f}$",
        # units are (axes-fraction, data)
        xy=(0.95, offset + 0.5),
        xycoords=ax.get_yaxis_transform(),
        # set the text alignment
        ha="right",
        va="bottom",
    )
    return {"raw": ln, "fit": fit_ln, "annotation": ann}



Which modifying our standard example to do the fit and pass the result into `plot_one`

In [ ]:
fig, ax = plt.subplots()
for j, indx in enumerate([6, 0, -1]):
    plot_one(ax, d[indx], fit(d[indx]), j*4, line_style={'linewidth': .75})
ax.set_xlabel('time (ms)')
ax.set_ylabel('displacement [offset] (mm)')
ax.legend();

Before we address the issue with the legend, we should talk about the choice of doing the fit inside or outside of `plot_one` and the more structured return type.

On one hand, if we made the call to `fit` inside of `plot_one` it would not require any changes to the API of `plot_one`!  Instead of "plot the experimental data" it would now have the job of "plot the experimental data and the best-fit curve".  This is a very reasonable extension of the scope of `plot_one` and would make sure that the fit and the raw-data were always correctly associated, however doing so would mix analysis and visualization in a fundamentally inseparable way. If we wanted to use the fit parameters in some other part of the code we would have the choice of either re-calling `fit` or returning the values of the fit from `plot_one`. 

If we were to re-call `fit` multiple places in the code it would waste time (you may have noted a bit of lag in running the plotting cell just above) and would present a risk that different parts of the code would be calling different versions of `fit` (or for techniques that may not be fully deterministic get slightly different results) which would lead to inconsistencies.  Consider if we wrote a `fit_but_better`, then we would have to find _every_ place in our code that we called `fit` and change it.

To avoid having to re-call `fit`, we could have `plot_one` return the fit values, however, this would plotting a pre requisite for fitting!  Consider the case were we had hundreds of experiments, it would be farcical to have to plot (and then discard the figure) every data set to extract all of the fit parameters!

Thus, we have chosen to pass the fit results into `plot_one`.